# MDP Obj 3

## Imports

In [1]:
import tensorflow as tf
import pandas as pd # for data manipulation 
import numpy as np
import math, os
from scipy import special

def euclidean_dist(row):
    # Function to calc euclidean distance on every df row 
    euc_dist = math.sqrt(row["U2G_Distance"]**2 - row["Height"]**2)
    return euc_dist

def q_func(x):
    q = 0.5 - 0.5*special.erf(x / np.sqrt(2))
    return q

def friis_calc(P,freq,dist,ple):
    '''
    Friis path loss equation
    P = Tx transmit power
    freq = Signal frequency
    dist = Transmission distance
    ple = Path loss exponent
    '''
    propagation_speed = 299792458
    l = propagation_speed / freq
    h_pl = P * l**2 / (16*math.pi**2)
    P_Rx = h_pl * dist**(-ple)
    return P_Rx

def plos_calc(h_dist, height_tx, height_rx, env='suburban'):
    '''
    % This function implements the LoS probability model from the paper
    % "Blockage Modeling for Inter-layer UAVs Communications in Urban
    % Environments" 
    % param h_dist    : horizontal distance between Tx and Rx (m)
    % param height_tx : height of Tx
    % param height_rx : height of Rx
    '''
    if env == 'suburban':
        a1 = 0.1
        a2 = 7.5e-4
        a3 = 8
    elif env == 'urban':
        a1 = 0.3
        a2 = 5e-4
        a3 = 15
    
    delta_h = height_tx - height_rx
    # pow_factor = 2 * h_dist * math.sqrt(a1*a2/math.pi) + a1 # NOTE: Use this pow_factor if assuming PPP building dist.
    pow_factor = h_dist * math.sqrt(a1*a2) # NOTE: Use this pow_factor if assuming ITU-R assumptions.
    if delta_h == 0:
        p = (1 - math.exp((-(height_tx)**2) / (2*a3**2))) ** pow_factor
    else:
        if delta_h < 0:
            h1 = height_rx
            h2 = height_tx
        else:
            h1 = height_tx
            h2 = height_rx
        delta_h = abs(delta_h)
        p = (1 - (math.sqrt(2*math.pi)*a3 / delta_h) * abs(q_func(h1/a3) - q_func(h2/a3))) ** pow_factor
    return p

def sinr_lognormal_approx(h_dist, height, env='suburban'):
    '''
    To approximate the SNR from signal considering multipath fading and shadowing
    Assuming no interference due to CSMA, and fixed noise
    Inputs:
    h_dist = Horizontal Distance between Tx and Rx
    height = Height difference between Tx and Rx
    env = The operating environment (currently only suburban supported)
    '''
    # Signal properties
    P_Tx_dBm = 20 # Transmit power of 
    P_Tx = 10**(P_Tx_dBm/10) / 1000
    freq = 2.4e9 # Channel frequency (Hz)
    noise_dBm = -86
    noise = 10**(noise_dBm/10) / 1000
    if env == "suburban":
        # ENV Parameters Constants ----------------------------------
        # n_min = 2
        # n_max = 2.75
        # K_dB_min = 7.8
        # K_dB_max = 17.5
        # K_min = 10**(K_dB_min/10)
        # K_max = 10**(K_dB_max/10)
        # alpha = 11.25 # Env parameters for logarithm std dev of shadowing 
        # beta = 0.06 # Env parameters for logarithm std dev of shadowing 
        n_min = 2
        n_max = 2.75
        K_dB_min = 1.4922
        K_dB_max = 12.2272
        K_min = 10**(K_dB_min/10)
        K_max = 10**(K_dB_max/10)
        alpha = 11.1852 # Env parameters for logarithm std dev of shadowing 
        beta = 0.06 # Env parameters for logarithm std dev of shadowing 
        # -----------------------------------------------------------
    elif env == "urban":
        n_min = 1.9
        n_max = 2.7
        K_dB_min = -5
        K_dB_max = 15
        K_min = 10**(K_dB_min/10)
        K_max = 10**(K_dB_max/10)
        alpha = 10.42 # Env parameters for logarithm std dev of shadowing 
        beta = 0.05 # Env parameters for logarithm std dev of shadowing 
    # Calculate fading parameters
    PLoS = plos_calc(h_dist, 0, height, env=env)
    theta_Rx = math.atan2(height, h_dist) * 180 / math.pi # Elevation angle in degrees
    ple = (n_min - n_max) * PLoS + n_max # Path loss exponent
    sigma_phi_dB = alpha*math.exp(-beta*theta_Rx)
    sigma_phi = 10**(sigma_phi_dB/10) # Logarithmic std dev of shadowing
    K = K_min * math.exp(math.log(K_max/K_min) * PLoS**2)
    omega = 1 # Omega of NCS (Rician)
    dist = math.sqrt(h_dist**2 + height**2)
    P_Rx = friis_calc(P_Tx, freq, dist, ple)
    # Approximate L-NCS RV (which is the SNR) as lognormal
    eta = math.log(10) / 10
    mu_phi = 10*math.log10(P_Rx)
    E_phi = math.exp(eta*mu_phi + eta**2*sigma_phi**2/2) # Mean of shadowing RV
    var_phi = math.exp(2*eta*mu_phi+eta**2*sigma_phi**2)*(math.exp(eta**2*sigma_phi**2)-1) # Variance of shadowing RV
    E_chi = (special.gamma(1+1)/(1+K))*special.hyp1f1(-1,1,-K)*omega
    var_chi = (special.gamma(1+2)/(1+K)**2)*special.hyp1f1(-2,1,-K)*omega**2 - E_chi**2
    E_SNR = E_phi * E_chi / noise # Theoretical mean of SINR
    var_SNR = ((var_phi+E_phi**2)*(var_chi+E_chi**2) - E_phi**2 * E_chi**2) / noise**2
    std_dev_SNR = math.sqrt(var_SNR)
    # sigma_ln = math.sqrt(math.log(var_SNR/E_SNR**2 + 1))
    # mu_ln = math.log(E_SNR) - sigma_ln**2/2
    return E_SNR, std_dev_SNR

def normalize_data(df, columns=[], save_details_path=None):
    '''
    columns: The pandas data columns to normalize, given as a list of column names
    '''
    # Define the ranges of parametrers
    max_mean_sinr = 10*math.log10(1123) # The max mean SINR calculated at (0,60) is 1122.743643457063 (linear)
    max_std_dev_sinr = 10*math.log10(466) # The max std dev SINR calculated at (0,60) is 465.2159856885714 (linear)
    min_mean_sinr = 10*math.log10(0.2) # The min mean SINR calculated at (1200,60) is 0.2251212887895188 (linear)
    min_std_dev_sinr = 10*math.log10(0.7) # The min std dev SINR calculated at (1200,300) is 0.7160093126585219 (linear)
    max_height = 300
    min_height = 60
    max_h_dist = 1200
    min_h_dist = 0

    # Normalize data (Min Max Normalization between [-1,1])
    if "Height" in columns:
        df["Height"] = df["Height"].apply(lambda x: 2*(x-min_height)/(max_height-min_height) - 1)
    if "U2G_H_Dist" in columns:
        df["U2G_H_Dist"] = df["U2G_H_Dist"].apply(lambda x: 2*(x-min_h_dist)/(max_h_dist-min_h_dist) - 1)
    if "Mean_SINR" in columns:
        df["Mean_SINR"] = df["Mean_SINR"].apply(lambda x: 2*(10*math.log10(x)-min_mean_sinr)/(max_mean_sinr-min_mean_sinr) - 1) # Convert to dB space
    if "Std_Dev_SINR" in columns:
        df["Std_Dev_SINR"] = df["Std_Dev_SINR"].apply(lambda x: 2*(10*math.log10(x)-min_std_dev_sinr)/(max_std_dev_sinr-min_std_dev_sinr) - 1) # Convert to dB space
    if "UAV_Sending_Interval" in columns:
        df["UAV_Sending_Interval"] = df["UAV_Sending_Interval"].replace({10:-1, 20:-0.5, 40:0, 66.7: 0.5, 100:1, 1000:2})
    if "Packet_State" in columns:
        df['Packet_State'] = df['Packet_State'].replace({"Reliable":0, "QUEUE_OVERFLOW":1, "RETRY_LIMIT_REACHED":2, "Delay_Exceeded":3})
    if "Modulation" in columns:
        df['Modulation'] = df['Modulation'].replace({"BPSK":1, "QPSK":0.3333, 16:-0.3333, "QAM-16":-0.3333, "QAM16":-0.3333, 64:-1, "QAM-64":-1, "QAM64":-1})

    # Record details of inputs and output for model
    if save_details_path is not None:
        f = open(os.path.join(save_details_path,"model_details.txt"), "w")
        f.write("Max Height (m): {}\n".format(max_height))
        f.write("Min Height (m): {}\n".format(min_height))
        f.write("Max H_Dist (m): {}\n".format(max_h_dist))
        f.write("Min H_Dist (m): {}\n".format(min_h_dist))
        f.write("Max Mean SINR (dB): {}\n".format(max_mean_sinr))
        f.write("Min Mean SINR (dB): {}\n".format(min_mean_sinr))
        f.write("Max Std Dev SINR (dB): {}\n".format(max_std_dev_sinr))
        f.write("Min Std Dev SINR (dB): {}\n".format(min_std_dev_sinr))
        f.write("[BPSK: 1, QPSK: 0.3333, QAM16: -0.3333, QAM64: -1]\n")
        f.write("UAV Sending Interval: [10:-1, 20:-0.5, 40:0, 100:0.5, 1000:1]\n")
        f.write("Output: ['Reliable':0, 'QUEUE_OVERFLOW':1, 'RETRY_LIMIT_REACHED':2, 'Delay_Exceeded':3]\n")
        f.close()

    return df

def norm_MCS(mcs_index):
    return 2*mcs_index/7 - 1

2023-10-30 23:37:26.807192: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 23:37:26.931103: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-30 23:37:26.935779: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-30 23:37:26.935794: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

## Load model

In [2]:
dl_model = tf.keras.models.load_model("/media/research-student/One Touch/FANET Datasets/Dataset_NP10000_DJISpark/nn_checkpoints/djispark_nnv4_wobn_dl/model.010-0.2039.h5", compile=False)
dl_model.compile(optimizer='adam', 
              loss={'packet_state': 'categorical_crossentropy'},
              metrics={'packet_state': 'accuracy'})

ul_model = tf.keras.models.load_model("/media/research-student/One Touch/FANET Datasets/Dataset_NP10000_DJISpark/nn_checkpoints/djispark_nnv4_wobn_ul/model.010-0.1202.h5", compile=False)
ul_model.compile(optimizer='adam', 
              loss={'packet_state': 'categorical_crossentropy'},
              metrics={'packet_state': 'accuracy'})

vid_model = tf.keras.models.load_model("/media/research-student/One Touch/FANET Datasets/Dataset_NP10000_DJISpark/nn_checkpoints/djispark_nnv4_wobn_vid/model.010-0.2581.h5", compile=False)
vid_model.compile(optimizer='adam', 
              loss={'packet_state': 'categorical_crossentropy'},
              metrics={'packet_state': 'accuracy'})

2023-10-30 23:37:32.932111: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-30 23:37:32.932259: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-10-30 23:37:32.932366: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-10-30 23:37:32.932468: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-10-30 23:37:32.932568: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

## Run MDP

## General Policy

In [5]:
def mdp(dl_model, ul_model, vid_model, reliability_th=0.99, heights=None, uav_send_ints=None, save_file=None):

    # SCENARIO: UAV swarm moving forward at fixed height
    # To get the max h_dist possible while maintaining reliability with height kep constant (MCS and USI optimized)

    # reliability_th = 0.99 # Target reliability 

    # Possible parameters:
    if heights == None:
        heights = [60, 90, 120, 150, 180, 210, 240, 270, 300]
    if uav_send_ints == None:
        uav_send_ints = [10, 20, 66.7, 100]
    mcs_indexes = [7,6,5,4,3,2,1,0]
    horizontal_dist = np.linspace(0, 1200, 121, endpoint=True)

    # Get normalized mean and std dev of sinr for each h_dist
    # mean_sinr = np.zeros(len(horizontal_dist))
    # std_dev_sinr = np.zeros(len(horizontal_dist))
    max_mean_sinr = 10*math.log10(1123) # The max mean SINR calculated at (0,60) is 1122.743643457063 (linear)
    max_std_dev_sinr = 10*math.log10(466) # The max std dev SINR calculated at (0,60) is 465.2159856885714 (linear)
    min_mean_sinr = 10*math.log10(0.2) # The min mean SINR calculated at (1200,60) is 0.2251212887895188 (linear)
    min_std_dev_sinr = 10*math.log10(0.7) # The min std dev SINR calculated at (1200,300) is 0.7160093126585219 (linear)

    uav_send_int_norm = {10:-1, 20:-0.5, 40:0, 66.7: 0.5, 100:1, 1000:2}
    max_h_dist = [] # To record the maximum h_dist under different heights

    # MDP 
    curr_uav_send_int_index = 0 # Index for UAV send int listed in variable uav_send_ints
    curr_height_index = 0
    curr_MCS = 7 # MCS Index
    state_record = []
    end = 0
    for j in range(0, len(horizontal_dist)): # Traverse the h_dist
        if end == 1:
            break
        # Get the order of closest heights and uav send ints based on current
        if len(heights) > 1:
            closest_height_indexes = [x for _,x in sorted(zip(np.abs(np.arange(len(heights))-curr_height_index), np.arange(len(heights))))][1:]
        else:
            closest_height_indexes = [] # Height is fixed in this case
        if len(uav_send_ints) > 1:
            closest_uav_send_int_indexes = [x for _,x in sorted(zip(np.abs(np.arange(len(uav_send_ints))-curr_uav_send_int_index), np.arange(len(uav_send_ints))))][1:]
        else:
            closest_uav_send_int_indexes = []
        next_height_index = curr_height_index # Initialize next_height_index
        next_uav_send_int_index = curr_uav_send_int_index # Initialize next_uav_send_int_index
        # First, check if next state is reliable
        m, s = sinr_lognormal_approx(horizontal_dist[j], heights[curr_height_index])
        mean_sinr = 2*(10*math.log10(m)-min_mean_sinr)/(max_mean_sinr-min_mean_sinr) - 1
        std_dev_sinr = 2*(10*math.log10(s)-min_std_dev_sinr)/(max_std_dev_sinr-min_std_dev_sinr) - 1
        next_dl_rel = dl_model.predict([[mean_sinr, std_dev_sinr, uav_send_int_norm[uav_send_ints[curr_uav_send_int_index]], norm_MCS(curr_MCS)]], verbose=0)[0][0]
        next_ul_rel = ul_model.predict([[mean_sinr, std_dev_sinr, uav_send_int_norm[uav_send_ints[curr_uav_send_int_index]], norm_MCS(curr_MCS)]], verbose=0)[0][0]
        next_vid_rel = vid_model.predict([[mean_sinr, std_dev_sinr, uav_send_int_norm[uav_send_ints[curr_uav_send_int_index]], norm_MCS(curr_MCS)]], verbose=0)[0][0]
        # If next state is not reliable, first explore different MCS before trying to change sending rate
        while (next_dl_rel < reliability_th) or (next_ul_rel < reliability_th) or (next_vid_rel < reliability_th):
            inputs = np.hstack((np.array([mean_sinr]*8).reshape(-1,1), np.array([std_dev_sinr]*8).reshape(-1,1), 
                                np.array([uav_send_int_norm[uav_send_ints[next_uav_send_int_index]]]*8).reshape(-1,1), np.array([norm_MCS(mcs) for mcs in mcs_indexes]).reshape(-1,1)))
            next_mcs_dl_results = dl_model.predict(inputs, verbose=0)
            next_mcs_dl_rel = [prob[0] for prob in next_mcs_dl_results]
            next_mcs_ul_results = ul_model.predict(inputs, verbose=0)
            next_mcs_ul_rel = [prob[0] for prob in next_mcs_ul_results]
            next_mcs_vid_results = vid_model.predict(inputs, verbose=0)
            next_mcs_vid_rel = [prob[0] for prob in next_mcs_vid_results]
            # Check if any MCS is able to maintain the reliability of all links
            next_mcs_rel_check = np.array([1 if rel >= reliability_th else 0 for rel in next_mcs_dl_rel]) & np.array([1 if rel >= reliability_th else 0 for rel in next_mcs_ul_rel]) & np.array([1 if rel >= reliability_th else 0 for rel in next_mcs_vid_rel]) 
            if not np.any(next_mcs_rel_check):
                if len(closest_height_indexes) == 0: # This means we've tried all heights
                    if len(closest_uav_send_int_indexes) == 0: # This means we've tried all UAV send ints
                        # If cannot increase the sending int further, and no MCS / height fulfills the requirements:
                        # Either just take best case for Downlink for continue (UNCOMMENT BELOW) ----------
                        # index = np.argmax(next_mcs_dl_rel)
                        # curr_MCS = mcs_indexes[index]
                        # next_dl_rel = next_mcs_dl_rel[index]
                        # next_ul_rel = next_mcs_ul_rel[index]
                        # next_vid_rel = next_mcs_vid_rel[index]
                        # Or stop the algorithm ------------------------------------------------------------
                        max_h_dist.append(horizontal_dist[j])
                        end = 1 # Stop the MDP
                        break
                    else:
                        # If none of the heights were able to satisfy reliability, we try to next closest UAV send int
                        # Reset the closest_height_indexes (NOTE: curr_height_index should not have been changed at this point from previous value)
                        if len(heights) > 1:
                            closest_height_indexes = [x for _,x in sorted(zip(np.abs(np.arange(len(heights))-curr_height_index), np.arange(len(heights))))][1:]
                        else:
                            closest_height_indexes = []
                        next_height_index = curr_height_index # Reset next_height_index
                        next_uav_send_int_index = closest_uav_send_int_indexes.pop(0) # Straight change the UAV send int
                        # print("Updating current send int: {}".format(curr_uav_send_int_index))
                        # Update SINR param based on current height
                        m, s = sinr_lognormal_approx(horizontal_dist[j], heights[curr_height_index])
                        mean_sinr = 2*(10*math.log10(m)-min_mean_sinr)/(max_mean_sinr-min_mean_sinr) - 1
                        std_dev_sinr = 2*(10*math.log10(s)-min_std_dev_sinr)/(max_std_dev_sinr-min_std_dev_sinr) - 1
                else:
                    # Try next closest height, update SINR params
                    next_height_index = closest_height_indexes.pop(0)
                    # print("Setting next height: {}".format(next_height_index))
                    m, s = sinr_lognormal_approx(horizontal_dist[j], heights[next_height_index])
                    mean_sinr = 2*(10*math.log10(m)-min_mean_sinr)/(max_mean_sinr-min_mean_sinr) - 1
                    std_dev_sinr = 2*(10*math.log10(s)-min_std_dev_sinr)/(max_std_dev_sinr-min_std_dev_sinr) - 1
            else:
                # Take highest MCS index that fulfills requirement
                index = next_mcs_rel_check.tolist().index(1) # The index returned should correspond to the correct MCS index
                curr_MCS = mcs_indexes[index]
                curr_height_index = next_height_index
                curr_uav_send_int_index = next_uav_send_int_index
                # print("Updating current height: {}".format(curr_height_index))
                next_dl_rel = next_mcs_dl_rel[index]
                next_ul_rel = next_mcs_ul_rel[index]
                next_vid_rel = next_mcs_vid_rel[index]
        
        # To record the optimal acctions under each height:
        state_record.append({"Horizontal_Distance": horizontal_dist[j], "Height": heights[curr_height_index], "UAV_Send_Interval": uav_send_ints[curr_uav_send_int_index], "MCS": curr_MCS, 
                            "DL_Reliability": next_dl_rel, "UL_Reliability": next_ul_rel, "Vid_Reliability": next_vid_rel})

    mdp_actions = pd.DataFrame(state_record)

    if save_file is not None:
        mdp_actions.to_csv(save_file)

    return mdp_actions

## Policy to keep Height Constant

In [ ]:
# SCENARIO: UAV swarm moving forward at fixed height
# To get the max h_dist possible while maintaining reliability with height kep constant (MCS and USI optimized)

reliability_th = 0.99 # Target reliability 

height = [60, 90, 120, 150, 180, 210, 240, 270, 300]
horizontal_dist = np.linspace(0, 1200, 121, endpoint=True)

# Get normalized mean and std dev of sinr for each h_dist
mean_sinr = np.zeros(len(horizontal_dist))
std_dev_sinr = np.zeros(len(horizontal_dist))
max_mean_sinr = 10*math.log10(1123) # The max mean SINR calculated at (0,60) is 1122.743643457063 (linear)
max_std_dev_sinr = 10*math.log10(466) # The max std dev SINR calculated at (0,60) is 465.2159856885714 (linear)
min_mean_sinr = 10*math.log10(0.2) # The min mean SINR calculated at (1200,60) is 0.2251212887895188 (linear)
min_std_dev_sinr = 10*math.log10(0.7) # The min std dev SINR calculated at (1200,300) is 0.7160093126585219 (linear)

uav_send_ints = {10:-1, 20:-0.5, 40:0, 66.7: 0.5, 100:1, 1000:2}
uav_send_ints_index = {0:10, 1:20, 2:66.7, 3:100, 4:1000}
max_uav_send_ints_index = 3
mcs_indexes = [7,6,5,4,3,2,1,0]
max_h_dist = [] # To record the maximum h_dist under different heights

for k in range(len(height)):
    # Get the mean and std dev of SINR for a certain height over the range of h_dist
    for i in range(len(horizontal_dist)):
        m, s = sinr_lognormal_approx(horizontal_dist[i], height[k])
        mean_sinr[i] = 2*(10*math.log10(m)-min_mean_sinr)/(max_mean_sinr-min_mean_sinr) - 1
        std_dev_sinr[i] = 2*(10*math.log10(s)-min_std_dev_sinr)/(max_std_dev_sinr-min_std_dev_sinr) - 1

    # MDP 
    curr_uav_send_int = 0 # Index for UAV send int
    curr_MCS = 7 # MCS Index
    state_record = []
    # state_record = [{"Horizontal_Distance": horizontal_dist[0], "Height": height, "UAV_Send_Interval": uav_send_ints_index[curr_uav_send_int], "MCS": curr_MCS, "Reliability": 1}]
    usi_record = [curr_uav_send_int] # To record USI, first entry at hdist=0m is initial value
    mcs_record = [curr_MCS] # To record MSC, first entry at hdist=0m is initial value
    go_to_next = 0
    for j in range(0, len(horizontal_dist)):
        if go_to_next == 1:
            break
        # First, check if next state is reliable
        next_dl_rel = dl_model.predict([[mean_sinr[j], std_dev_sinr[j], uav_send_ints[uav_send_ints_index[curr_uav_send_int]], norm_MCS(curr_MCS)]])[0][0]
        next_ul_rel = ul_model.predict([[mean_sinr[j], std_dev_sinr[j], uav_send_ints[uav_send_ints_index[curr_uav_send_int]], norm_MCS(curr_MCS)]])[0][0]
        next_vid_rel = vid_model.predict([[mean_sinr[j], std_dev_sinr[j], uav_send_ints[uav_send_ints_index[curr_uav_send_int]], norm_MCS(curr_MCS)]])[0][0]
        # If next state is not reliable, first explore different MCS before trying to change sending rate
        while (next_dl_rel < reliability_th) or (next_ul_rel < reliability_th) or (next_vid_rel < reliability_th):
            inputs = np.hstack((np.array([mean_sinr[j]]*8).reshape(-1,1), np.array([std_dev_sinr[j]]*8).reshape(-1,1), 
                                np.array([uav_send_ints[uav_send_ints_index[curr_uav_send_int]]]*8).reshape(-1,1), np.array([norm_MCS(mcs) for mcs in mcs_indexes]).reshape(-1,1)))
            next_mcs_dl_results = dl_model.predict(inputs)
            next_mcs_dl_rel = [prob[0] for prob in next_mcs_dl_results]
            next_mcs_ul_results = ul_model.predict(inputs)
            next_mcs_ul_rel = [prob[0] for prob in next_mcs_ul_results]
            next_mcs_vid_results = vid_model.predict(inputs)
            next_mcs_vid_rel = [prob[0] for prob in next_mcs_vid_results]
            # Check if any MCS is able to maintain the reliability of all links
            next_mcs_rel_check = np.array([1 if rel >= reliability_th else 0 for rel in next_mcs_dl_rel]) & np.array([1 if rel >= reliability_th else 0 for rel in next_mcs_ul_rel]) & np.array([1 if rel >= reliability_th else 0 for rel in next_mcs_vid_rel]) 
            if not np.any(next_mcs_rel_check):
                if curr_uav_send_int == max_uav_send_ints_index:
                    # If cannot increase the sending int further, and no MCS fulfills the requirements:
                    # Either just take best case for Downlink for continue (UNCOMMENT BELOW) ----------
                    # index = np.argmax(next_mcs_dl_rel)
                    # curr_MCS = mcs_indexes[index]
                    # next_dl_rel = next_mcs_dl_rel[index]
                    # next_ul_rel = next_mcs_ul_rel[index]
                    # next_vid_rel = next_mcs_vid_rel[index]
                    # Or stop the algorithm ------------------------------------------------------------
                    max_h_dist.append(horizontal_dist[j])
                    go_to_next = 1 # To break out of nested for loop
                    break
                # Increase UAV send int
                curr_uav_send_int += 1
            else:
                # Take highest MCS index that fulfills requirement
                index = next_mcs_rel_check.tolist().index(1) # The index returned should correspond to the correct MCS index
                curr_MCS = mcs_indexes[index]
                next_dl_rel = next_mcs_dl_rel[index]
                next_ul_rel = next_mcs_ul_rel[index]
                next_vid_rel = next_mcs_vid_rel[index]
        
        # To record the optimal acctions under each height:
        # state_record.append({"Horizontal_Distance": horizontal_dist[j], "Height": height, "UAV_Send_Interval": uav_send_ints_index[curr_uav_send_int], "MCS": curr_MCS, 
        #                     "DL_Reliability": next_dl_rel, "UL_Reliability": next_ul_rel, "Vid_Reliability": next_vid_rel})

    # mdp_actions = pd.DataFrame(state_record)
    # mdp_actions.to_csv("mdp_policy/constant_height_{}m_policy.csv".format(height))

### Single Scenario Save Policy

In [6]:
uav_send_ints = [100]
heights = [120]
mdp_actions = mdp(dl_model, ul_model, vid_model, reliability_th=0.99, heights=heights, uav_send_ints=uav_send_ints, save_file="mdp_policy/constant_height_120m_constant_usi_100ms_policy.csv")

## Policy to keep UAV Sending Interval Constant

In [ ]:
uav_send_ints = [10, 20, 66.7, 100]
heights = [60, 90, 120, 150, 180, 210, 240, 270, 300]
max_h_dist = []
for send_int in uav_send_ints:
    mdp_actions = mdp(dl_model, ul_model, vid_model, reliability_th=0.99, heights=heights, uav_send_ints=[send_int])
    max_h_dist.append(mdp_actions["Horizontal_Distance"].max())

In [57]:
# Test model single scenario
height = 60
horizontal_dist = 10
mcs_index = 5
uav_send_int = 66.7
uav_send_int_norm = {10:-1, 20:-0.5, 40:0, 66.7: 0.5, 100:1, 1000:2}
max_mean_sinr = 10*math.log10(1123) # The max mean SINR calculated at (0,60) is 1122.743643457063 (linear)
max_std_dev_sinr = 10*math.log10(466) # The max std dev SINR calculated at (0,60) is 465.2159856885714 (linear)
min_mean_sinr = 10*math.log10(0.2) # The min mean SINR calculated at (1200,60) is 0.2251212887895188 (linear)
min_std_dev_sinr = 10*math.log10(0.7) # The min std dev SINR calculated at (1200,300) is 0.7160093126585219 (linear)
m, s = sinr_lognormal_approx(horizontal_dist, height)
m = 2*(10*math.log10(m)-min_mean_sinr)/(max_mean_sinr-min_mean_sinr) - 1
s = 2*(10*math.log10(s)-min_std_dev_sinr)/(max_std_dev_sinr-min_std_dev_sinr) - 1
result = dl_model.predict([[m,s,uav_send_int_norm[uav_send_int],norm_MCS(mcs_index)]])
result

1/1 [==============================] - 0s 35ms/step


array([[9.9997270e-01, 3.8384448e-19, 1.3092136e-32, 2.7313077e-05]],
      dtype=float32)